<a href="https://colab.research.google.com/github/Sc0rpie/VU_DeepLearning/blob/main/Lab1_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pirmas laboratorinis darbas

Olen Račkauskas, 3 grupė, 2 pogrupis, 2213787 (VGG16)

Klasifikuojamos klasės: Orange, Banana, Strawberry

In [1]:
!pip install openimages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 3.0 MB/s eta 0:00:00


In [2]:
import os
import torch
import torch.nn as nn
import numpy as np
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

Atsiunčiamos nuotraukos (300 Orange ir Banana, 400 Strawberry = 1000)

In [3]:
data_dir = "data"
number_for_samples = 300
classes_to_download = ["Orange", "Banana"]
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

print("Download commencing...")
from openimages.download import download_dataset
download_dataset(data_dir, classes_to_download, limit=number_for_samples)
download_dataset(data_dir, ["Strawberry"], limit=400)
print("Download finished.")

Download commencing...


100%|██████████| 400/400 [00:10<00:00, 39.43it/s]


Download finished.


Sukomponuojam transformacijas bei panaudojam jau aprašyta DataSet: ImageFolder

Prie to pačio patikrinam ar turim 1000 nuotraukų

In [4]:
all_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [12]:
from torch.utils.data import Dataset
from PIL import Image
class MyDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.samples = []

        self.classes = sorted(
            [d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]
        )

        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

        for cls in self.classes:
            cls_folder = os.path.join(root, cls)
            for root_dir, _, files in os.walk(cls_folder):
                for file in files:
                    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                        path = os.path.join(root_dir, file)
                        self.samples.append((path, self.class_to_idx[cls]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        image_path, label = self.samples[index]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
dataset = MyDataset(root=data_dir, transform=all_transforms)
print(f"Dataset size: {len(dataset)}")

Dataset size: 1000


Paruošiamas DataLoader bei peržiūrimas klasių mapping'as

In [7]:
loader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2)

classes = sorted(os.listdir('data'))
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
print("Class to index mapping:", class_to_idx)

Class to index mapping: {'banana': 0, 'orange': 1, 'strawberry': 2}


Paruošiamas VGG16 modelis naudojimui

In [8]:
model = models.vgg16(pretrained=True)
model.to("cuda")
model.eval();

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 161MB/s]


Aprašomos klasių ID iš imagenet 1000 klasių sąrašo

In [9]:
dataset_to_imagenet = {
    0: 954,   # "Banana"
    1: 950,   # "Orange"
    2: 949    # "Strawberry"
}

Aprašomos funkcijos skaičiavimams

In [10]:
def compute_metrics(ground_truth, predictions, threshold=0.5):
    pred_bin = (predictions > threshold).astype(np.float64)
    TP = np.sum((ground_truth == 1) & (pred_bin == 1))
    TN = np.sum((ground_truth == 0) & (pred_bin == 0))
    FP = np.sum((ground_truth == 0) & (pred_bin == 1))
    FN = np.sum((ground_truth == 1) & (pred_bin == 0))
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    return {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'f1': f1}

def get_class_predictions(prob_array, labels, imagenet_class_index, dataset_label):
    gt = (labels == dataset_label).astype(np.float64)
    pred = prob_array[:, imagenet_class_index]
    return gt, pred

Paleidžiamas modelis, įrašomos visos tikimybės į masyvą

In [13]:
all_probabilities = []
all_labels = []

for images, labels in loader:
    images, labels = images.cuda(), labels.cuda()
    outputs = model(images)
    probabilities = torch.sigmoid(outputs).detach().cpu().numpy()
    all_probabilities.append(probabilities)
    all_labels.append(labels.cpu().numpy())

all_probabilities = np.concatenate(all_probabilities, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

Gauti rezultatai naudojami skaičiavimams ir rezultato atvaizdavimui

In [14]:
metrics_results = {}
for dataset_label, imagenet_idx in dataset_to_imagenet.items():
    gt, pred = get_class_predictions(all_probabilities, all_labels, imagenet_idx, dataset_label)
    metrics = compute_metrics(gt, pred, threshold=0.95)
    class_name = list(class_to_idx.keys())[list(class_to_idx.values()).index(dataset_label)]
    metrics_results[class_name] = metrics

for class_name, metrics in metrics_results.items():
    print(f"{class_name} metrics: {metrics}")

banana metrics: {'accuracy': 0.333, 'recall': 0.9766666666666667, 'precision': 0.3074501573976915, 'f1': 0.4676775738228252}
orange metrics: {'accuracy': 0.35, 'recall': 0.97, 'precision': 0.31223175965665234, 'f1': 0.4724025974025974}
strawberry metrics: {'accuracy': 0.618, 'recall': 0.9925, 'precision': 0.5115979381443299, 'f1': 0.6751700680272107}


Nuotrauku patikrinimui

In [ ]:
from PIL import Image

def predict_images(image_paths, model, transform, device="cuda"):
    if isinstance(image_paths, str):
        image_paths = [image_paths]

    model.eval()
    predictions = []

    for path in image_paths:
        try:
            img = Image.open(path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {path}: {e}")
            continue

        input_tensor = transform(img).unsqueeze(0).to(device)

        # Inference.
        with torch.no_grad():
            outputs = model(input_tensor)
            probs = torch.sigmoid(outputs).cpu().numpy()[0]

        predictions.append(probs)

    return predictions

In [ ]:
import numpy as np

def print_top_predictions(pred_vector, topk=5):
    top_indices = np.argsort(pred_vector)[::-1][:topk]
    print("Top predictions:")
    for idx in top_indices:
        print(f"  Class id {idx}: probability {pred_vector[idx]:.4f}")

In [ ]:
image_files = ["data/gmm.jpg"]
preds = predict_images(image_files, model, all_transforms, device="cuda")
print_top_predictions(preds[0], topk=5)